In [36]:
!wget -O food11.zip https://www.dropbox.com/s/up5q1gthsz3v0dq/food-11.zip?dl=0

--2025-06-24 14:31:28--  https://www.dropbox.com/s/up5q1gthsz3v0dq/food-11.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/w5r49vs5e956w6c4z6ob9/food-11.zip?rlkey=3no5l2xjiqgk2ckwbewaanm5p&dl=0 [following]
--2025-06-24 14:31:29--  https://www.dropbox.com/scl/fi/w5r49vs5e956w6c4z6ob9/food-11.zip?rlkey=3no5l2xjiqgk2ckwbewaanm5p&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9acc397263d128af52c90b3e36.dl.dropboxusercontent.com/cd/0/inline/CsP2Ikssx5JEk22nW0z-xKFoAS1bg7KgrkWU1itrQCP9CKSe3Dvagh3z0P2s_prnXhXJW3dA5d7qnZYnwRBWG57471HPKQuOUhjRk_yRx5Yr99P_g7_9VylRdXOvgznXfMDySBrNrNpGoe0dOCs1BrKh/file# [following]
--2025-06-24 14:31:29--  https://uc9acc397263d128af52c90b3e36.dl.dropboxusercontent.com/c

In [37]:
!unzip -o /kaggle/working/food11.zip -d /kaggle/working/

Archive:  /kaggle/working/food11.zip
   creating: /kaggle/working/valid/
  inflating: /kaggle/working/valid/9_2898.jpg  
  inflating: /kaggle/working/valid/9_3053.jpg  
  inflating: /kaggle/working/valid/9_294.jpg  
  inflating: /kaggle/working/valid/9_2048.jpg  
  inflating: /kaggle/working/valid/9_3190.jpg  
  inflating: /kaggle/working/valid/9_1020.jpg  
  inflating: /kaggle/working/valid/9_2861.jpg  
  inflating: /kaggle/working/valid/9_3012.jpg  
  inflating: /kaggle/working/valid/9_723.jpg  
  inflating: /kaggle/working/valid/9_2008.jpg  
  inflating: /kaggle/working/valid/9_2755.jpg  
  inflating: /kaggle/working/valid/9_3225.jpg  
  inflating: /kaggle/working/valid/9_976.jpg  
  inflating: /kaggle/working/valid/9_2319.jpg  
  inflating: /kaggle/working/valid/9_2961.jpg  
  inflating: /kaggle/working/valid/9_1706.jpg  
  inflating: /kaggle/working/valid/9_1504.jpg  
  inflating: /kaggle/working/valid/9_698.jpg  
  inflating: /kaggle/working/valid/9_218.jpg  
  inflating: /kaggle

In [ ]:
_exp_name = "sample"

In [ ]:
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
from tqdm.auto import tqdm
import random

In [ ]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

In [ ]:
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])
train_tfm = transforms.Compose([
    transforms.RandomHorizontalFlip(), # 随即水平翻转
    transforms.RandomRotation(15), # 随机旋转
    transforms.ColorJitter(brightness = 0.2)
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [ ]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
            
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
            
        return im,label

通道数增加：提取更复杂的特征
尺寸逐步减小：聚焦关键区域
卷积核大小：常用3x3(兼顾局部和计算效率),kernel_size = 3
    卷积核大小：小目标用3x3,大目标用5x5
通道数：按2的倍数增加(经验法则) 64->128->256->512
    通道数：每层x2,直到显存不足
步长/填充:保持尺寸用1/1,减半用2/0 stride = 1,padding=1
   步长：1(保细节)/2(降维)      填充：0/1 padding = kernel_size//2
全连接层维度：通常在512~4096之间试验，1024->512->11

卷积层堆叠：模拟人脑从整体到局部的识别过程
池化层：让模型不在乎物体的位置（上下左右都一样）
通道增加：有更多的检测点(有的专查纹理，有的专查颜色)
这个神经网络是CNN的经典套路，适合中小型图像分类任务(Kaggle比赛)，如果是更复杂的任务比如医学影像，就需要更深或者更特殊的结构

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
 
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0), 

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Classifier().to(device)

batch_size = 64
n_epochs = 8
patience = 5 # 早停

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)
# 再次使用Adam，考虑了历史梯度方向，形成惯性调参只需要调lr，其他参数用默认值
# Adam适用于大多数深度学习任务并且更快更稳真乃神器也！

In [ ]:
train_set = FoodDataset("/kaggle/working/train", tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = FoodDataset("/kaggle/working/valid", tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

In [ ]:
stale = 0
best_acc = 0

for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):

        imgs, labels = batch

        logits = model(imgs.to(device))
        loss = criterion(logits, labels.to(device))
        optimizer.zero_grad()
        
        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
        optimizer.step()
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        train_loss.append(loss.item())
        train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------

    model.eval()
    valid_loss = []
    valid_accs = []

    for batch in tqdm(valid_loader):

      
        imgs, labels = batch

        with torch.no_grad():
            logits = model(imgs.to(device))


        loss = criterion(logits, labels.to(device))

        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        valid_loss.append(loss.item())
        valid_accs.append(acc)

    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

    if valid_acc > best_acc:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    else:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

    if valid_acc > best_acc:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break

In [ ]:
test_set = FoodDataset("/kaggle/working/test", tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

In [ ]:
model_best = Classifier().to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
model_best.eval()
prediction = []
with torch.no_grad():
    for data,_ in tqdm(test_loader):
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()

In [ ]:
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(len(test_set))]
df["Category"] = prediction
df.to_csv("submission.csv",index = False)

In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import os
from tqdm.auto import tqdm
import numpy as np


def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(6666)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 数据增强和预处理
train_tfm = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_tfm = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 自定义数据集类
class FoodDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        
        try:
            # 假设文件名格式为"类别_编号.jpg"，如"3_0001.jpg"表示类别3
            label = int(self.image_files[idx].split('_')[0])
        except:
            label = -1  # 测试集无标签
            
        if self.transform:
            image = self.transform(image)
            
        return image, label


train_dir = "/kaggle/working/train" 
valid_dir = "/kaggle/working/valid" 
test_dir = "/kaggle/working/test"  

train_dataset = FoodDataset(train_dir, transform=train_tfm)
valid_dataset = FoodDataset(valid_dir, transform=test_tfm)
test_dataset = FoodDataset(test_dir, transform=test_tfm)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

Using device: cuda


In [44]:
# 初始化ResNet模型
def get_resnet_model(num_classes=11):
    model = models.resnet50(pretrained=True)  # 使用预训练的ResNet50
    
    # 冻结所有卷积层参数（可选）
    for param in model.parameters():
        param.requires_grad = False
        # 冻结：将卷积层的 requires_grad 属性设为 False，使这些层在反向传播时不计算梯度，权重不会被优化器更新。
    # 修改最后一层全连接层
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, num_classes)
    )
    return model

model = get_resnet_model(num_classes=11).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
# 动态学习率调整策略（学习率调度器）
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2, factor=0.5)
"""
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,   # 绑定优化器（如Adam/SGD）,一般都用Adam，唯一真神
    mode='max',  # 监控指标的方向（'max'表示希望指标越大越好）
    patience=2,  # 容忍轮数（连续2次指标未提升则降低学习率） 早停
    factor=0.5   # 学习率衰减系数（每次降低为原来的0.5倍）
)
"""

def train_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Validating"):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


n_epochs = 15
patience = 5
best_acc = 0.0
stale = 0

for epoch in range(n_epochs):
    print(f"\nEpoch {epoch+1}/{n_epochs}")
    
 
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    

    val_loss, val_acc = validate(model, valid_loader, criterion)
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    

    scheduler.step(val_acc)
    
 
    if val_acc > best_acc:
        print(f"Validation accuracy improved from {best_acc:.4f} to {val_acc:.4f}")
        best_acc = val_acc
        torch.save(model.state_dict(), "best_resnet_model.pth")
        stale = 0
    else:
        stale += 1
        print(f"No improvement for {stale}/{patience} epochs")
        if stale >= patience:
            print("Early stopping")
            break


print("\nLoading best model for testing...")
model.load_state_dict(torch.load("best_resnet_model.pth"))
model.eval()

predictions = []
with torch.no_grad():
    for images, _ in tqdm(test_loader, desc="Testing"):
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())


def pad4(i):
    return "0"*(4-len(str(i))) + str(i)

df = pd.DataFrame({
    "Id": [pad4(i) for i in range(len(test_dataset))],
    "Category": predictions
})
df.to_csv("submission.csv", index=False)
print("预测结果已保存到 submission.csv")


Epoch 1/3


Training:   0%|          | 0/157 [00:00<?, ?it/s]

Train Loss: 1.6628 | Train Acc: 0.4839


Validating:   0%|          | 0/57 [00:00<?, ?it/s]

Val Loss: 1.0364 | Val Acc: 0.7137
Validation accuracy improved from 0.0000 to 0.7137

Epoch 2/3


...

Train Loss: 0.9308 | Train Acc: 0.7115


Validating:   0%|          | 0/57 [00:00<?, ?it/s]

Val Loss: 0.6696 | Val Acc: 0.7941
Validation accuracy improved from 0.7809 to 0.7941

Loading best model for testing...


Testing:   0%|          | 0/47 [00:00<?, ?it/s]

预测结果已保存到 submission.csv


这里是因为训练到12轮的时候掉了， 所以补上剩下的3轮，实际上是15轮

In [33]:
pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.
